## CMBAgent - OSS Testing

*For testing purposes only, to be removed before opening a PR to* `main` .

In [ ]:
# Local model testing supported via https://github.com/acceleratescience/mini-vllm (private Accelerate repo)

import os
from dotenv import load_dotenv

load_dotenv()

BASE_URL = os.getenv("LOCAL_LLM_BASE_URL")
API_KEY = os.getenv("LOCAL_LLM_API_KEY")

local_llm_config = {
    "config_list": [
        {
            "model": "openai/gpt-oss-20b", # Must match model id from endpoint
            "api_key": API_KEY,
            "api_type": "openai",
            "base_url": BASE_URL,
            "price": [0.0, 0.0], # Passed explicitly to ignore warning
        },
    ],
    "cache_seed": None
}

### AG2 ConversableAgent() with `openai/gpt-oss-20b`

Simple chat with a tiny model (`openai/gpt-oss-20b`)to verify ConversableAgent() has OSS

In [ ]:
from autogen import UserProxyAgent, ConversableAgent

# Creates the agent that uses the vLLM LM
assistant = ConversableAgent(
    "agent",
    llm_config=local_llm_config,
    system_message="Answer briefly.",
)

# Create the agent that represents the user in the conversation
user_proxy = UserProxyAgent(
    "user",
    code_execution_config=False,
    system_message="",
    human_input_mode="NEVER",
)

# User asks a question
assistant.initiate_chat(
    user_proxy,
    message="What is the meaning of Denario?",
    max_turns=2,
)

agent (to user):

What is the meaning of Denario?

--------------------------------------------------------------------------------
user (to agent):



--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
             Model agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
openai/gpt-oss-20b agent 0.00000             86                219           305
agent (to user):

**Denario** (from Latin *denarius*) is a silver coin that served as the basic unit of currency in ancient Rome. It was a common everyday piece of money used for trade and paying wages. In later periods the word also came to refer broadly to a small amount of money.

--------------------------------------------------------------------------------
user (to agent):



--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (25601e1c-8e68-4851-a444-81ca04a96109): Maximum turns (2) reached


ChatResult(chat_id=7595919412255891713458123805495119465, chat_history=[{'content': 'What is the meaning of Denario?', 'role': 'assistant', 'name': 'agent'}, {'content': '', 'role': 'user', 'name': 'user'}, {'content': '**Denario** (from Latin *denarius*) is a silver coin that served as the basic unit of currency in ancient Rome. It was a common everyday piece of money used for trade and paying wages. In later periods the word also came to refer broadly to a small amount of money.', 'role': 'assistant', 'name': 'agent'}, {'content': '', 'role': 'user', 'name': 'user'}], summary='', cost={'usage_including_cached_inference': {'total_cost': 0.0, 'openai/gpt-oss-20b': {'cost': 0.0, 'prompt_tokens': 86, 'completion_tokens': 219, 'total_tokens': 305}}, 'usage_excluding_cached_inference': {'total_cost': 0.0, 'openai/gpt-oss-20b': {'cost': 0.0, 'prompt_tokens': 86, 'completion_tokens': 219, 'total_tokens': 305}}}, human_input=[])

### AG2 ConversableAgent() Tool Calling with OSS

In [ ]:
def add(a: float, b: float) -> float:
    return a + b

assistant.register_for_llm(name="add", description="Add two numbers")(add)
assistant.register_for_execution(name="add")(add)

result = assistant.initiate_chat(
    recipient=assistant,
    message="Use the add tool to add 2 and 3. Then reply with just the number.",
    max_turns=3,
)

for msg in result.chat_history:
    if msg.get("tool_calls"):
        print(msg["tool_calls"])

agent (to agent):

Use the add tool to add 2 and 3. Then reply with just the number.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
             Model agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
openai/gpt-oss-20b agent 0.00000            175                 35           210
agent (to agent):

***** Suggested tool call (chatcmpl-tool-a10236a53e17d3d0): add *****
Arguments: 
{"a": 2, "b": 3}
*********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION add...
Call ID: chatcmpl-tool-a10236a53e17d3d0
Input arguments: {'a': 2, 'b': 3}

>>>>>>>> EXECUTED FUNCTION add...
Call ID: chatcmpl-tool-a10236a53e17d3d0
Input arguments: {'a': 2, 'b': 3}
Output:
5.0
agent (to agent):

***** Response from calling Tool tool (chatcmpl-tool-a10236a53e17d3d0) *****
5.0
**********

### OSS with Engineer Only

In [ ]:
from pathlib import Path
from cmbagent.workflows import one_shot

local_engineer_config = local_llm_config["config_list"][0]

from dotenv import load_dotenv

load_dotenv()

tmp_path = Path.cwd()

task = """
Plot the graph of y = x from x = 0 to 1
"""

results = one_shot(
    task,

    max_rounds=5,
    agent="engineer",
    engineer_model=local_engineer_config,
    work_dir=str(tmp_path / "one_shot"),
)

_User (to main_cmbagent_chat):


Plot the graph of y = x from x = 0 to 1


--------------------------------------------------------------------------------

Next speaker: engineer


>>>>>>>> USING AUTO REPLY...
             Model    agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
openai/gpt-oss-20b engineer 0.00000           1768               1488          3256
engineer (to main_cmbagent_chat):

**Code Explanation:**

The script generates a simple linear plot of y = x over the interval [0, 1]. It creates a `data` directory if it does not exist, constructs the plot with Matplotlib, and saves it as a high‑resolution PNG file (300 dpi) in that directory. The filename includes a timestamp to ensure uniqueness. After saving, the script prints a concise description of the plot and the file path.

**Modifications:**  
None.

**Python Code:**

```python
#!/usr/bin/env python3
"""
Plot y = x from 0 to 1 and save to data folder.
"""

import os
import datetime
import numpy as np
im

NotFoundError: Error code: 404 - {'error': {'message': 'The model `o3-mini-2025-01-31` does not exist.', 'type': 'NotFoundError', 'param': 'model', 'code': 404}}

Next step: add OSS for other engineering agents, e.g. `engineer_response_formatter`